In [55]:
import numpy as np
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage import io
import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [57]:
# Constants
IMAGE_SIZE = (128, 128)  # Target size for resized images
DATA_PATH = './train2/*/*.jpg'  # File path pattern to locate dataset

In [59]:
def preprocess_image(image_path, target_size=IMAGE_SIZE):
    image = io.imread(image_path)  # Read the image
    gray_image = rgb2gray(image)  # Convert to grayscale
    resized_image = resize(gray_image, target_size, anti_aliasing=True)  # Resize to target size
    normalized_image = resized_image / 255.0  # Normalize pixel values to [0, 1]
    return normalized_image.flatten()  # Flatten into a 1D array

In [61]:
def load_data(data_path):
    data, labels = [], []  # Initialize lists for data and labels
    image_paths = glob.glob(data_path)  # Get all image paths matching the pattern
    
    for image_path in tqdm(image_paths, desc="Processing Images"):
        data.append(preprocess_image(image_path))  # Preprocess and add to the data list
        labels.append(1 if 'cat' in image_path.lower() else 0)  # Assign labels based on the file name
    
    return np.array(data), np.array(labels)  # Return data and labels as NumPy arrays

In [63]:
data, labels = load_data(DATA_PATH)  # Load data using the defined function
print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")  # Print dataset shapes

Processing Images: 0it [00:00, ?it/s]

Data shape: (0,), Labels shape: (0,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

In [ ]:
clf = SVC(kernel='linear', random_state=42)  # Create an SVM classifier with a linear kernel
clf.fit(X_train, y_train)  # Train the model on the training data
print("Model training completed!")

In [ ]:
y_pred = clf.predict(X_test)  # Predict labels for the test data
print("\nClassification Report:")
print(classification_report(y_test, y_pred))  # Print precision, recall, and f1-score
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))  # Print confusion matrix

In [ ]:
def predict_and_display(image_path, model):
    processed_image = preprocess_image(image_path)  # Preprocess the test image
    prediction = model.predict([processed_image])[0]  # Predict the class of the image
    
    # Display the image with its predicted label
    plt.imshow(resize(io.imread(image_path), IMAGE_SIZE, anti_aliasing=True), cmap='gray')
    plt.title(f'Predicted class is {"Cat" if prediction == 1 else "Dog"}')
    plt.axis('off')
    plt.show()

In [ ]:
test_image_path = './train2/dog/dog.108.jpg'  # Path to a test image
predict_and_display(test_image_path, clf)  # Predict and display the test image